<center> <h1> Homework 1 </h1> </center>

### Taylor Patti
### 2/7/2016








## Exercises Completed:

1. Exercise 3.8 (```adaptive_trapzint.py```)
1. Exercise 3.15 (```sinesum1.py```)
1. Exercise 3.18 (```centered_diff.py```)
1. Exercise 3.20 (```find_primes.py```)




## adaptive_trapzint.py
#### Uses algorithm to calculate the number of series terms required to obtain integral value within a given error, then computes this quantity. A select sampling of functions, intervals,  and function preformance is output below.

In [38]:
# %load adaptive_trapzint.py
"""
File: adaptive_trapzint.py

Copyright (c) 2016 Taylor Patti

License: MIT

This code establishes and compares functions of various complexities which use trapezoids
to estimate the area under curves in the plane.
""" 

import math
from sympy import symbols, integrate, lambdify

def trapzint(f, a, b, n):
    """Estimates integral by the construction of n evenly spaced trapezoids on the interval
    [a, b]"""
    h = (b-a) / float(n)
    sum = 0
    for i in range(n):
        #h serves double duty as both the x-distance width and the x-distance increment
        sum = sum + (1.0/2)*h*(f(a + h*i) + f(a + h*(i + 1)))
    return sum

def square(t):
    return 2

def test_square():
    """A two by two square is measured and should have area 4."""
    apt = (abs(trapzint(square, 0, 2, 20)-4) < 1E-6)
    msg = "Not square area."
    assert apt, msg

def test_sinusoidal():
    """Ensures that the area of sin from 0 to pi/2 is 1."""
    apt = (abs(trapzint(math.sin, 0, math.pi / 2, 1000)-1)<1E-6)
    msg = "Not sin area."
    assert apt, msg

def second_derivative(f, x, h=1e-6):
    return (f(x - h) - 2 * f(x) + f(x + h)) / float(h**2)

def max_second_derivative(f, a, b, n=1000000):
    """Obtains the maximum absolute value of the second derivative of a function
    at n points."""
    inc = (b - a) / float(n)
    max = abs(second_derivative(f, a))
    for i in range(n):
        candidate = abs(second_derivative(f, a + inc * (i + 1)))
        if candidate > max:
            max = candidate
    return max

def test_sin_maxderivative():
    """sin's max derivative should be 1"""
    apt = (abs(max_second_derivative(math.sin, 0, 2 * math.pi) - 1) < 1E-3)
    msg = "Wrong derivative for sin"
    assert apt, msg
    

def adaptive_trapzint(f, a, b, eps=1E-5):
    """Calculates the number of intervals required to obtain integral estimation within
    some precision and implements trapzint with this value."""
    h = math.sqrt(12 * eps) * (((b - a) * max_second_derivative(f, a, b))**(-1 / 2.0))
    n = (b-a)/float(h)
    #ceil taken to ensure the integrity of the inequality
    n = int(math.ceil(n))
    return trapzint(f, a, b, n), n

def curve(x):
    return x**2 + x + 1

def test_adaptive_limits():
    """ensures that the n value given is sufficient for the curve function.
    This function has calculated n of 366 on the interval 0 to 2"""
    t = symbols('t')
    curve_func = t**2 + t + 1
    integ = integrate(curve_func, t)
    value = lambdify([t], integ)
    value = value(2)
    print value
    apt = abs(adaptive_trapzint(curve, 0, 2)[0]-value)<1E-5
    msg = "N is too small."
    assert apt, msg

def preformance_table(function_tuples=[('cos(x)', math.cos, [0.0, math.pi]), \
                                       ('sin(x)', math.sin, \
    [0.0, math.pi]), ('sin(x)', math.sin, [0, math.pi/2.0])]):
    """Outputs the function, interval, error, and n values into a table. Note that here,
    the exact answers were simply given in an array as these integrals were considered
    both common and fundamental knowledge. If you would like to see an example of
    symbolic integration and subsequent numerical evaluation, please see
    above the function 'test_adaptive_limits'"""
    exact = [0, 2, 1]
    print
    print 'Preformance Table of Various Functions'
    print
    print '%10s %15s %15s %15s' % ('Function', 'Interval', 'Error', 'N-value')
    print
    for i in function_tuples:
        value, nvalue = adaptive_trapzint(i[1], i[2][0], i[2][1])
        error = value-exact[function_tuples.index(i)]
        print '%9s %10.2f %2s %3.2f %17E %9d' % (i[0], i[2][0], 'to', i[2][1], error, nvalue)
        print

preformance_table()



Preformance Table of Various Functions

  Function        Interval           Error         N-value

   cos(x)       0.00 to 3.14      3.122502E-17       509

   sin(x)       0.00 to 3.14     -6.349115E-06       509

   sin(x)       0.00 to 1.57     -6.346204E-06       180



## sinesum1.py

#### Constructs a trigonometric series with a known point of conversion and ultimately produces a table which details the effectiveness of truncating at various indexes for a sampling of arguments.

In [39]:
# %load sinesum1.py
"""
File: sinesum1.py

Copyright (c) 2016 Taylor Patti

License: MIT

Sums sines and compares them to a step function.
"""    

from math import *

def S(t, n, T):
    sum = 0
    for i in range(n):
        sum = sum + ((2 * (i + 1) - 1)**-1) * sin((2 * (2 * (i + 1) - 1) \
        * pi * t) / float(T))
    return sum*(4/pi)

def test_twosinesum():
    """checks that the S function is commensurate with direct calculation
    for t = 1, n = 2, T = 2pi"""
    apt = abs(S(1, 2, 2 * pi) - (4 / pi) * (sin(1) + sin(3) / 3)) < 1E-3
    msg = "Sums do not match."
    assert apt, msg
    
def f(t, T):
    if (t > 0) and (t < (T / float(2))):
        return 1
    elif t == T / float(2):
        return 0
    elif (t > (T / float(2))) and (t < T):
        return -1

def test_tequalTover2():
    apt = f(0.5, 1) == 0
    msg = "Equality wrong"
    assert apt, msg
    
def test_tgreaterTover2():
    apt = f(0.75, 1) == -1
    msg = "Greater than wrong."
    assert apt, msg
    
def difference(t, n, T):
    return f(t, T) - S(t, n, T)
    
def table(t):
    print
    print 'Difference Between the Limit and the Sum of Sines'
    print
    print '%10s %10s %10s %10s' % ('n', 'a=0.01', 'a=0.25', 'a=0.49')
    print
    n_list = [1, 3, 5, 10, 30, 100]
    alpha_list = [0.01, 0.25, 0.49]
    T = 2 * pi
    alpha1 = [difference(t*alpha_list[0], n_value, T) for n_value in n_list]
    alpha2 = [difference(t*alpha_list[1], n_value, T) for n_value in n_list]
    alpha3 = [difference(t*alpha_list[2], n_value, T) for n_value in n_list]
    for i in range(6):
        print '%10d %10.2f %10.2f %10.2f' % (n_list[i], alpha1[i], alpha2[i], alpha3[i])
        
table(1)





Difference Between the Limit and the Sum of Sines

         n     a=0.01     a=0.25     a=0.49

         1       0.99       0.68       0.40
         3       0.96       0.15      -0.18
         5       0.94      -0.14       0.00
        10       0.87       0.01      -0.06
        30       0.63      -0.03      -0.01
       100      -0.02       0.01      -0.01


## centered_diff.py

#### Estimates the first derivative of a function by a truncated Taylor Series approximation. Ultimately, a table was produced in which the estimations were compared to the analytic derivatives, calculated using
#### sympy, and the error was deduced.

In [40]:
# %load centered_diff.py
"""
File: centered_diff.py

Copyright (c) 2016 Taylor Patti

License: MIT

Takes the first derivative of functions numerically and demonstrates
this with a set of functions and a resulting table.
"""    

from math import *
from sympy import diff, lambdify, symbols

def numdiff(f, x, h=1E-5):
    """takes the first derivative of function f"""
    return (f(x + h) - f(x - h)) / (2 * float(h))

def quadratic(x):
    return x**2 + x + 1

def test_quaddiff():
    apt = (abs(numdiff(quadratic, 2) - 5) < 1E-10)
    msg = "Derivatives do not match."
    assert apt, msg

def application():
    """makes list of the difference between the estimated derivative for h=0.01
    and the analytical derivative"""
    application_list = []
    x = symbols('x')
    print
    print '%20s %20s' % ('Function', 'Error')
    print
    funcvar_tuples = [(lambda t: exp(t), 'exp(x)', float(0)), (lambda t : exp(-2*t**2), \
    'exp(-2*x**2)', float(0)), (lambda t: cos(t), 'cos(x)', 2*pi), (lambda t: log(t), \
    'log(x)', float(1))]
    for i in funcvar_tuples:
        prime = diff(i[1], x)
        exact = lambdify([x], prime)
        error = numdiff(i[0], i[2], 0.01) - exact(i[2])
        print '%20s %21.2E' % (i[1], error)
        print
        application_list.append(error)
    return application_list

application()

def test_accuracycheck():
    """ensures that each item in application is bellow a certain threshold
    and thus above a certain accuracy"""
    f = application()
    apt = True
    for i in f:
        if abs(i) > 1E-3:
            apt = False
    msg = "Too large of an error."
    assert apt, msg



            Function                Error

              exp(x)              1.67E-05

        exp(-2*x**2)              0.00E+00

              cos(x)             -2.45E-16

              log(x)              3.33E-05



## find_primes.py

#### This program identifies prime factors by iterating through a list from 2 to the number given as the function argument and eliminating numbers which are multiples of smaller numbers from the iteration list.
#### The iteration list is then returned and consists of only prime numbers lesser or equal to the function argument. Here, the prime numbers of 100 and less are shown.

In [41]:
# %load find_primes.py
"""
File: find_primes.py

Copyright (c) 2016 Taylor Patti

License: MIT

Finds primes by elimination of multiples of smaller values in and below a given value
n, then returns a list of the prime numbers.
"""    

def find_primes(n):
    """finds all of the prime numbers of n and below by eliminating mutiples of
    smaller numbers from a master list"""
    num_list = range(n + 1)
    num_list = num_list[2:]
    for i in num_list:
        p = i + i
        while p <= max(num_list):
            if p in num_list:
                num_list.remove(p)
            p = p + i
    return num_list

def test_firstfourprimes():
    """ensures that the list produced is equal to [2, 3, 5, 7], the prime numbers
    corresponding to the value n=10"""
    apt = find_primes(10) == [2, 3, 5, 7]
    msg = "These are not the primes you're looking for."
    assert apt, msg
    
print find_primes(100)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
